In [1]:
!git clone https://github.com/deep-learning-with-pytorch/dlwpt-code.git

Cloning into 'dlwpt-code'...
remote: Enumerating objects: 703, done.
remote: Total 703 (delta 0), reused 0 (delta 0), pack-reused 703
Receiving objects: 100% (703/703), 176.00 MiB | 18.63 MiB/s, done.
Resolving deltas: 100% (309/309), done.
Updating files: 100% (228/228), done.


In [2]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [3]:
import csv
wine_path = "/content/dlwpt-code/data/p1ch4/tabular-wine/winequality-white.csv"   #資料路徑
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [4]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))   #取得標題列所有特徵名稱

wineq_numpy.shape, col_list   #輸出wineq_numpy陣列的shape及所有特徵名稱

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [5]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [6]:
data = wineq[:, :-1]  #除了最後一行其他資料都選擇
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [7]:
target = wineq[:, -1]   #取出wineq品質分數做為目標張量
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [8]:
target = wineq[:, -1].long()    #將wineq中的品質分數轉為整數
target

tensor([6, 6,  ..., 7, 6])

In [10]:
target_onehot = torch.zeros(target.shape[0], 10)    #建立shape為[4898,10]且值均為0的張量，存放編碼結果

a = target_onehot.scatter_(1, target.unsqueeze(1), 1.0)   #依target內容進行編碼

a, a.shape

(tensor([[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         ...,
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]]),
 torch.Size([4898, 10]))

In [11]:
target_unsqueezed = target.unsqueeze(1)   #增加一軸
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [12]:
data_mean = torch.mean(data, dim=0)   #計算平均值並指定從第0軸計算
data_mean

tensor([6.85e+00, 2.78e-01, 3.34e-01, 6.39e+00, 4.58e-02, 3.53e+01,
        1.38e+02, 9.94e-01, 3.19e+00, 4.90e-01, 1.05e+01])

In [13]:
data_var = torch.var(data, dim=0)     #計算標準差並指定從第0軸計算
data_var

tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
        1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00])

In [14]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.72e-01, -8.18e-02,  ..., -3.49e-01, -1.39e+00],
        [-6.57e-01,  2.16e-01,  ...,  1.34e-03, -8.24e-01],
        ...,
        [-1.61e+00,  1.17e-01,  ..., -9.63e-01,  1.86e+00],
        [-1.01e+00, -6.77e-01,  ..., -1.49e+00,  1.04e+00]])

In [15]:
bad_indexes = target <= 3     #挑出target張量中分數小於或等於3分的項目
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [16]:
bad_data = data[bad_indexes]    #利用索引篩選出符合目標的項目
bad_data.shape

torch.Size([20, 11])

In [17]:
bad_data = data[target <= 3]    #篩選-差
mid_data = data[(target > 3) & (target < 7)]    #篩選-中等
good_data = data[target >= 7]   #篩選-好

bad_mean = torch.mean(bad_data, dim=0)      #差的各化學性質品均值
mid_mean = torch.mean(mid_data, dim=0)      #中等的各化學性質品均值
good_mean = torch.mean(good_data, dim=0)    ##好的各化學性質品均值

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [18]:
total_sulfur_threshold = 141.83   #設定閥值為141.83，若超出此標準則為品質差的酒，低於則為好的酒
total_sulfur_data = data[:,6]   #取出資料集的總二氧化硫量資訊
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)   #挑出A中值小於B的項目索引

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()   #輸出小於閥值的樣本總數(好酒)

(torch.Size([4898]), torch.bool, tensor(2727))

In [19]:
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()    #輸出品質分數大於5的樣本總數

(torch.Size([4898]), torch.bool, tensor(3258))

In [20]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()    #實際好酒與預測好酒項目索引是否吻合
n_predicted = torch.sum(predicted_indexes).item()   #預測為好酒的項目數量
n_actual = torch.sum(actual_indexes).item()         #實際為好酒的項目數量

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)